In [1]:
import os
import yaml
import multiprocessing
import itertools

import pypsa

import numpy as np
import pandas as pd

ERROR 1: PROJ: proj_create_from_database: Open of /fp/homes01/u01/ec-koenvg/.conda/envs/my_base/envs/eu-hydrogen/share/proj failed


In [2]:
config_fn = "../config/build-year-aggregation-validation.yaml"
config = yaml.safe_load(open(config_fn))

config_default = yaml.safe_load(open("../config/config.default.yaml"))

results_dir = "../results"

In [3]:
stat_names = [
    "h2prod",
    "imports",
    "system_cost",
    "obj_bound_dual",
    "h2price",
    "co2seq",
    "dac",
    "captured_co2",
    "elec_cap",
    "elec_cf",
    "total_onwind",
    "total_offwind",
    "total_solar",
]


def stats(n: pypsa.Network):
    # H2
    i = n.links.loc[n.links.carrier.isin(["H2 Electrolysis"])].index
    MWh_h2 = -(
        n.links_t.p1.loc[:, i].sum(axis=1) * n.snapshot_weightings.generators
    ).sum()
    h2prod = (MWh_h2 / 33.33) / 1e6

    # Imports
    i = n.generators.loc[n.generators.index.str.contains("green import")].index
    imports = (
        n.generators_t.p.loc[:, i].sum(axis=1) * n.snapshot_weightings.generators
    ).sum() / 1e6

    # System cost
    system_cost = n.statistics.capex().sum() + n.statistics.opex().sum()

    # System cost bound dual
    obj_bound_dual = (
        n.global_constraints.at["total_system_cost", "mu"]
        if "total_system_cost" in n.global_constraints.index
        else np.nan
    )

    # H2 price
    h2_buses = n.buses.index[n.buses.carrier == "H2"]
    h2price = (n.buses_t.marginal_price[h2_buses].mean(axis=1) * n.snapshot_weightings.generators).sum() / 8760

    # Total CO2 sequestration
    co2seq = n.stores.loc[n.stores.carrier == "co2 sequestered", "e_nom_opt"].sum()

    # Total amount of CO2 captured by DAC
    dac_i = n.links.loc[n.links.carrier == "DAC"].index
    dac = (
        n.links_t.p0.loc[:, dac_i].sum(axis=1) * n.snapshot_weightings.generators
    ).sum()

    # Total amount of CO2 captured, including industry, BECCS, etc.
    total = 0
    for i in [1, 2, 3, 4]:
        links_i = n.links.loc[n.links.__getattr__(f"bus{i}") == "co2 stored"].index
        co2 = n.links_t.__getattr__(f"p{i}").loc[:, links_i]
        co2 = co2.clip(upper=0)
        total -= (co2.sum(axis=1) * n.snapshot_weightings.generators).sum()

    captured_co2 = total / 1e6

    # Total installed capacity of electrolysers
    elec_i = n.links.loc[n.links.carrier == "H2 Electrolysis"].index
    elec_cap = n.links.loc[elec_i, "p_nom_opt"].sum()

    # Mean capacity factor of electrolysers
    if elec_cap > 0:
        elec_cf = (
            n.links_t.p0.loc[:, elec_i].sum(axis=1) * n.snapshot_weightings.generators
        ).sum() / (elec_cap * n.snapshot_weightings.generators.sum())
    else:
        elec_cf = np.nan

    # Total installed onshore wind
    total_onwind = n.generators.loc[n.generators.carrier == "onwind", "p_nom_opt"].sum()

    # Total installed offshore wind
    total_offwind = n.generators.loc[
        n.generators.carrier.isin(["offwind-ac", "offwind-dc", "offwind-float"]),
        "p_nom_opt",
    ].sum()

    # Total installed solar
    total_solar = n.generators.loc[n.generators.carrier == "solar", "p_nom_opt"].sum()

    return {
        "h2prod": h2prod,
        "imports": imports,
        "system_cost": system_cost,
        "obj_bound_dual": obj_bound_dual,
        "h2price": h2price,
        "co2seq": co2seq,
        "dac": dac,
        "captured_co2": captured_co2,
        "elec_cap": elec_cap,
        "elec_cf": elec_cf,
        "total_onwind": total_onwind,
        "total_offwind": total_offwind,
        "total_solar": total_solar,
    }

In [4]:
nets = {}

scenario_list = [config["run"]["name"]]

# Assume that the following are uniquely defined for each run:
ll = config["scenario"]["ll"][0]
clusters = config["scenario"]["clusters"][0]
opts = config["scenario"]["opts"][0]

# The following are taken to vary for each run.
sector_opts = config["scenario"]["sector_opts"]
planning_horizons = config["scenario"]["planning_horizons"]
slacks = config["scenario"]["slack"]
senses = ["min", "max"]

def load_stats(file):
    return stats(pypsa.Network(file)) if os.path.exists(file) else {v: np.nan for v in stat_names}


index = list(itertools.product(scenario_list, sector_opts, planning_horizons, slacks, senses))
index_opt = list(itertools.product(scenario_list, sector_opts, planning_horizons, [0], ["opt"]))

with multiprocessing.Pool(30) as pool:
    networks = pool.map(
        load_stats,
        [
            f"{results_dir}/{s}/postnetworks/base_s_{clusters}_l{ll}_{opts}_{o}_{h}_{sense}{slack}.nc"
            for s, o, h, slack, sense in index
        ],
    )

    networks.extend(
        pool.map(
            load_stats,
            [
                f"{results_dir}/{s}/postnetworks/base_s_{clusters}_l{ll}_{opts}_{o}_{h}.nc"
                for s, o, h, _, _ in index_opt
            ],
        )
    )

networks = dict(zip(index + index_opt, networks))

INFO:pypsa.io:Imported network base_s_60_lc1.5__Cb-Ib-Ea-100seg_2040_min0.05.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_60_lc1.5__Ca-Ia-Ea-100seg_2025_min0.05.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_60_lc1.5__Cc-Ib-Ea-100seg_2040_min0.05.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_60_lc1.5__Ca-Ia-Eb-100seg_2025_min0.05.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_60_lc1.5__Cb-Ia-Eb-100seg_2025_min0.05.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_60_lc1.5__Cc-Ia-Ea-100seg_2025_min0.05.nc has buses, carriers, generators, global_

In [5]:
df = pd.DataFrame(networks).T.reset_index()
df.columns = ["scenario", "opts", "horizon", "slack", "sense"] + stat_names

# Strip any suffix of the form "-\d+seg" from the "opts" column
df["opts"] = df["opts"].str.replace(r"-\d+seg", "", regex=True)

# The "opts" column has the form "XX-YY-ZZ-..."; we want to split this into separate columns and keep only the last character
flags = ["C", "I", "E", "buildyearagg"]
df[flags] = (
    df["opts"]
    .str.split("-", expand=True)
    .map(
        lambda s: (
            s[-1]
            if (isinstance(s, str) and len(s) == 2)
            else ("a" if s else "b") # a for 0.2, b for default
        )
    )
)
df.drop(columns=["opts"], inplace=True)

# Now set the index to the flags, slack and horizon
df.set_index(flags + ["sense", "slack", "horizon"], inplace=True)

# Drop scenario column
df.drop(columns=["scenario"], inplace=True)

display(df)

h2prod       imports   system_cost  \
C I E buildyearagg sense slack horizon                                          
a a a b            min   0.05  2025     -0.000000  3.600438e-11  9.815186e+11   
                   max   0.05  2025      9.477340  8.315572e-09  1.018326e+12   
                   min   0.05  2030     -0.000000  4.570343e-11  9.370238e+11   
                   max   0.05  2030     11.370998  6.358150e-08  9.745048e+11   
                   min   0.05  2035     -0.000000  1.430109e-10  9.170501e+11   
...                                           ...           ...           ...   
c b b a            opt   0.00  2030      0.000028  1.467284e-11  9.287458e+11   
                               2035      2.812273  5.016464e+01  8.375699e+11   
                               2040     23.454208  1.892683e+02  8.937686e+11   
                               2045     20.034358  1.907000e+02  8.304796e+11   
                               2050     17.927838  2.957248e+02  7.677431e+11   

                                        obj_bound_dual     h2price  \
C I E buildyearagg sense slack horizon                               
a a a b            min   0.05  2025                NaN   58.942119   
                   max   0.05  2025         -13.034608    0.595170   
                   min   0.05  2030                NaN   86.468252   
                   max   0.05  2030         -17.256829    0.942510   
                   min   0.05  2035                NaN  205.606258   
...                                                ...         ...   
c b b a            opt   0.00  2030                NaN   73.508902   
                               2035                NaN   88.743156   
                               2040                NaN   88.796458   
                               2045                NaN   64.861083   
                               2050                NaN   64.428523   

                                              co2seq           dac  \
C I E buildyearagg sense slack horizon                               
a a a b            min   0.05  2025     0.000000e+00  1.235610e-01   
                   max   0.05  2025    -5.721655e-01  1.017059e-02   
                   min   0.05  2030     2.500000e+07  1.350647e-01   
                   max   0.05  2030     2.500000e+07  8.620641e-03   
                   min   0.05  2035     7.500000e+07  1.335391e-01   
...                                              ...           ...   
c b b a            opt   0.00  2030     1.000000e+08  2.171352e-06   
                               2035     3.000000e+08  1.250604e-01   
                               2040     5.000000e+08  1.084467e+08   
                               2045     5.580307e+08  1.180442e+08   
                               2050     5.957680e+08  1.156769e+08   

                                        captured_co2       elec_cap   elec_cf  \
C I E buildyearagg sense slack horizon                                          
a a a b            min   0.05  2025         7.154780       0.000000       NaN   
                   max   0.05  2025         7.154780   61388.205411  0.999997   
                   min   0.05  2030        32.154780       0.000000       NaN   
                   max   0.05  2030        52.909721   77671.996155  0.936164   
                   min   0.05  2035        82.154780       0.000000       NaN   
...                                              ...            ...       ...   
c b b a            opt   0.00  2030       107.154780       0.303285  0.563198   
                               2035       307.154780   30596.200297  0.548667   
                               2040       617.217012  229182.825385  0.597671   
                               2045       647.276055  229178.899972  0.510066   
                               2050       667.917815  229178.899965  0.456378   

                                         total_onwind  total_offwind  \
C I E buildyearagg sense slack horizon   

# Statistics

In [ ]:
# Error in percentage
errors = {
    attr: 100
    * (df.xs("a", level="buildyearagg") - df.xs("b", level="buildyearagg"))
    .abs()[attr]
    / df[attr].abs().mean()
    for attr in sorted(list(set(stat_names) - {"obj_bound_dual", "h2price"}))
}

pd.DataFrame(errors).reset_index(drop=True).mean().round(2)

captured_co2     0.05
co2seq           0.01
dac              0.20
elec_cap         0.01
elec_cf          0.28
h2prod           0.04
imports          0.35
system_cost      0.01
total_offwind    0.03
total_onwind     0.01
total_solar      0.01
dtype: float64